In [1]:
import pandas as pd
import numpy as np

In [2]:
# create the casenumbers for Sarah's pipeline:
# Specimen date, hosp, Area code, Area name, Area type
df_rki=pd.read_csv("/project_data/data_asset/sun/casenumbers/rki_covid19.csv")

In [3]:
#Perform the logic check to get all new cases:
df_rki_temp = df_rki[((df_rki["NeuerFall"]==0) | (df_rki["NeuerFall"]==1))]

In [4]:
# Do some data cleaning
# In RKI data Berlin is divided in 12 separate regions, but for displaying the regions, these need to be summed up
lk=[i for i in df_rki_temp["Landkreis"].unique() if "Berlin" in i]
df_berlin_old=df_rki_temp[df_rki_temp["Landkreis"].isin(lk)] 
df_berlin_new=df_berlin_old.groupby("Meldedatum",as_index=False)[["AnzahlFall"]].sum()
df_berlin_new["Landkreis"]="Berlin, Stadt"
# drop old berlin rows
df_rki_temp.drop(df_rki_temp[df_rki_temp["Landkreis"].isin(lk)].index,axis=0,inplace=True)
# add new berlin rows
df_rki_temp=pd.concat([df_rki_temp,df_berlin_new])

/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [5]:
# get the data for all Landkreise:
df_pipeline_germany_lk=pd.DataFrame(columns=["Landkreis","AnzahlFall","Landkreis cumsum"])
df_rki_lk=df_rki_temp.groupby(["Landkreis","Meldedatum"],as_index=False)[["AnzahlFall"]].sum()
for i in df_rki_lk["Landkreis"].unique():
    df=df_rki_lk[df_rki_lk["Landkreis"]==i]
    df.set_index("Meldedatum", inplace=True, drop=True)
    df.index=pd.to_datetime(df.index,format="%Y-%m-%d")
    df.sort_index(inplace=True)
    df["Landkreis cumsum"]=np.cumsum(df["AnzahlFall"])
    df_pipeline_germany_lk=pd.concat([df_pipeline_germany_lk,df])

/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [6]:
# get the data for all Bundesländer
df_pipeline_germany_bl=pd.DataFrame(columns=["Bundesland","AnzahlFall","Bundesland cumsum"])
df_rki_bl=df_rki_temp.groupby(["Bundesland","Meldedatum"],as_index=False)[["AnzahlFall"]].sum()
for i in df_rki_bl["Bundesland"].unique():
    df=df_rki_bl[df_rki_bl["Bundesland"]==i]
    df.set_index("Meldedatum", inplace=True, drop=True)
    df.index=pd.to_datetime(df.index,format="%Y-%m-%d")
    df.sort_index(inplace=True)
    df["Bundesland cumsum"]=np.cumsum(df["AnzahlFall"])
    df_pipeline_germany_bl=pd.concat([df_pipeline_germany_bl,df])

/opt/conda/envs/Python-3.6/lib/python3.6/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# new columns for Country and Area Type
df_pipeline_germany_lk["Area type"]="Landkreis"
df_pipeline_germany_lk["Country"]="Germany"
df_pipeline_germany_bl["Area type"]="Bundesland"
df_pipeline_germany_bl["Country"]="Germany"

In [8]:
# change the column names
df_pipeline_germany_bl.columns=["Area name","new cases","cum cases","Area type","Country"]
df_pipeline_germany_lk.columns=["Area name","new cases","cum cases","Area type","Country"]

In [9]:
# read mapping table
mapping=pd.read_csv("/project_data/data_asset/mapping_rki_cognos.csv")

In [10]:
mapping[mapping["rki name"]=="Berlin, Stadt"]

,rki name,cognos name,Cca 2
400,"Berlin, Stadt","Berlin, Stadt",11000


In [12]:
# add columns for cognos names and id

for i in mapping["rki name"]:
    new_name=mapping[mapping["rki name"]==i]["cognos name"].values[0]
    new_id=mapping[mapping["rki name"]==i]["Cca 2"].values[0]
    df_pipeline_germany_lk.loc[df_pipeline_germany_lk["Area name"]==i,"Cognos Area name"]=new_name
    df_pipeline_germany_lk.loc[df_pipeline_germany_lk["Area name"]==i,"Cca 2"]=new_id

In [13]:
df_pipeline_germany_bl["Cognos Area name"]=df_pipeline_germany_bl["Area name"]
df_pipeline_germany_bl["Cca 2"]=np.nan

In [14]:
# combine into one Dataframe
df_pipeline_germany=pd.concat([df_pipeline_germany_lk,df_pipeline_germany_bl])

In [15]:
df_pipeline_germany.head(10)

,Area name,new cases,cum cases,Area type,Country,Cognos Area name,Cca 2
2020-03-03,"Berlin, Stadt",6,6,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-04,"Berlin, Stadt",3,9,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-05,"Berlin, Stadt",10,19,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-06,"Berlin, Stadt",5,24,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-07,"Berlin, Stadt",1,25,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-08,"Berlin, Stadt",11,36,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-09,"Berlin, Stadt",22,58,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-10,"Berlin, Stadt",34,92,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-11,"Berlin, Stadt",42,134,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-12,"Berlin, Stadt",47,181,Landkreis,Germany,"Berlin, Stadt",11000.0


In [16]:
# do some validation of the numbers
# stated or today cases in Germany: 195228
print(len(df_rki_temp),len(df_pipeline_germany),len(df_pipeline_germany_bl),len(df_pipeline_germany_lk))
print(min(df_pipeline_germany_bl.index),max(df_pipeline_germany_bl.index),(max(df_pipeline_germany_bl.index)-
                                                                           min(df_pipeline_germany_bl.index)))
print(sum(df_pipeline_germany["new cases"])/2)

154957 29297 1972 27325
2020-01-28 00:00:00 2020-07-18 00:00:00 172 days 00:00:00
197187.0


In [17]:
# save the dataframe
df_pipeline_germany.to_csv("/project_data/data_asset/df_pipeline_germany.csv",index_label=False)

In [19]:
# sanity check
df_test=pd.read_csv("/project_data/data_asset/df_pipeline_germany.csv")
df_test.head(5)

,Area name,new cases,cum cases,Area type,Country,Cognos Area name,Cca 2
2020-03-03,"Berlin, Stadt",6,6,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-04,"Berlin, Stadt",3,9,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-05,"Berlin, Stadt",10,19,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-06,"Berlin, Stadt",5,24,Landkreis,Germany,"Berlin, Stadt",11000.0
2020-03-07,"Berlin, Stadt",1,25,Landkreis,Germany,"Berlin, Stadt",11000.0


In [20]:
df_test["Area type"].unique()

array(['Landkreis', 'Bundesland'], dtype=object)